In [29]:
import numpy as np
import scipy as sp
from scipy.fftpack import fft
from numba import njit

In [30]:
@njit
def stft(x, win, step):
    l = len(x) # 入力信号の長さ
    N = len(win) # 窓幅、つまり切り出す幅
    M = int(np.ceil(float(l - N + step) / step)) # スペクトログラムの時間フレーム数
    
    new_x = np.zeros(N + ((M - 1) * step),dtype="float64")
    new_x[: l] = x # 信号をいい感じの長さにする
    
    X = np.zeros([M, N], dtype = "complex64") # スペクトログラムの初期化(複素数型)
    for m in range(M):
        start = step * m
        X[m, :] = sp.fft.fft(new_x[start : start + N] * win)
    return X

@njit
def istft(X, win, step):
    M, N = X.shape
    assert (len(win) == N), "FFT length and window length are different."

    l = (M - 1) * step + N
    x = np.zeros(l,dtype="float64")
    wsum = np.zeros(l,dtype="float64")
    for m in range(M):
        start = step * m
        ### 滑らかな接続
        x[start : start + N] = x[start : start + N] + sp.fft.ifft(X[m, :]).real * win
        wsum[start : start + N] += win ** 2 
    pos = (wsum != 0)
    x_pre = x.copy()
    ### 窓分のスケール合わせ
    x[pos] /= wsum[pos]
    return x

In [33]:
from IPython.display import display, Audio
import librosa
import glob


def main():
    
    c_files = glob.glob("/disk107/Datasets/CMU_ARCTIC/*/*wav/*.wav")
    data ,fs = librosa.load(str(c_files[0]),sr = None)
    data = data[:2**15]
    display(Audio(data[:2**15] ,rate=fs))

    fftLen = 512 # とりあえず
    win = np.hamming(fftLen) # ハミング窓
    step = int(fftLen / 2)

    ### STFT
    spectrogram = stft(data[:2**15], win, step)

    ### iSTFT
    resyn_data = istft(spectrogram, win, step)

    display(Audio(resyn_data ,rate=fs))

    print(spectrogram.shape)

In [34]:
main()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function zeros>) with argument(s) of type(s): (int64, dtype=Literal[str](float64))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<built-in function zeros>)
[2] During: typing of call at <ipython-input-30-518e9ed99f6c> (7)


File "<ipython-input-30-518e9ed99f6c>", line 7:
def stft(x, win, step):
    <source elided>
    
    new_x = np.zeros(N + ((M - 1) * step),dtype="float64")
    ^
